## Part 1

In [6]:
import glob 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#getting the directory for the stored CSV's for the flow only case
csvdir = "C:\\Users\\Harrison\\OneDrive - University of Cincinnati\\Year 5\\Aero Lab\\Lab Work\\Lab 4\\Lab Data\\Flow only, vectors"

#importing the csv files
csvfiles = glob.glob(os.path.join(csvdir, "*.csv"))

#creating an array to store the imported csv dataframes
dataframes = []

for csvfile in csvfiles:
    df = pd.read_csv(csvfile) #reading each csv file
    dataframes.append(df) #adding the dataframe from the csv into the dataframes array

The cylinder and wing are placed in the freestream where the effects from the walls (bottom of tunnel, top of water) are the lowest, therefore we want to use that velocity to determine Reynold's number. For each frame, every row (y position) will be averaged and then all the frame rows will be averaged (gives v_avg row 1, v_avg row 2, etc.). Plotting the data will show which rows are the highest, and therefore the freestream. The freestream rows will then be averaged and the resulting speed will be used in the Re number calculation.

In [3]:
avg_speed = []
row = 0
print(len(dataframes))

for k in range(len(dataframes)):
    df = dataframes[k] #getting individual frame data
    speeds = []
    speed_arr = []

    for i in range(len(df)): 
        u = df.loc[i][2] #getting u component
        v = df.loc[i][3] #getting v component
        speed = np.sqrt(u**2 + v**2) #calculating speed for the position
        speed_arr.append(speed) #adding the speed value to the array

    avg_speed.append(np.average(speed_arr)) #average speed for the frame

    #counter to help track progress
    if k/100 == int(k/100):
        print("Frame {}".format(k))


print(avg_speed) #printing the avg speed array
flow_speed = np.average(avg_speed) #calculating the average flow speed
print(flow_speed) #displaying average flow speed


497
Frame 0
Frame 100
Frame 200
Frame 300
Frame 400
[130.3548471191971, 126.15378728424561, 132.26757131617072, 131.58078558260743, 130.36974698168484, 133.8948458231916, 126.10830908896192, 131.65680222803297, 131.87974640730744, 129.93911455982615, 129.54904677530115, 129.64044950609906, 124.33031740642686, 121.34575243040983, 128.22559724311532, 132.40908866533752, 129.06469459011294, 128.36724322595086, 126.2744023979616, 120.51773955080462, 123.96702874779533, 133.29055398835223, 127.21012894239747, 131.66495039534564, 129.32719080333177, 131.68971155020233, 129.81296783023308, 130.18470210399386, 129.15898591807107, 128.95937585412594, 128.154539390093, 128.7923291656114, 126.6139467583996, 128.384450493172, 130.984318441854, 125.15926993435778, 127.6651241089155, 128.64474941467776, 127.72587213817698, 126.30643926147206, 127.74174808080602, 119.94125050881074, 128.12948004850207, 126.83108396834007, 127.68372911680338, 126.93364789031672, 130.74776830207205, 127.28791887012214,

In [4]:
#constants
rho = 997 #kg/m^3
mu = 1.002E-3 #Ns/m^2
flow_speed = np.average(avg_speed)/1000 #m/s
r_c = 101.6 #mm
r_t = 57.66 #mm
Cyl_dia = 1.5/39.37 #m

t = (r_t/r_c) #non-dimensional, taper ratio
MAC = r_c * .66 * ((1+t+t**2)/(1+t))/1000 #m, mean aerodynamic chord

Re_cyl = (rho*flow_speed*Cyl_dia)/mu
Re_MAC = (rho*flow_speed*MAC)/mu

print("Re_cyl: {:.3f} \nRe_MAC: {:.3f}".format(Re_cyl, Re_MAC))

Re_cyl: 4520.760 
Re_MAC: 9591.350


# Part 2
A python function will take in the instantaneous vector fields for each image and then average the entire window to get the average vector field. The averaged mean turbulence RMS will also be computed 

In [5]:
#thought out method:
## Part A: Compiling all the data
# 1. Open the folder that contains the csv files
# 2. Open each csv file and then place the data into a dataframe and then into an array
# 3. Generate a panda's data frame (rows = frame #, columns = vector at position) [refer as total frame]

## Part B: Determining the mean velocity field
# 4. Calculate the vector for each position (per frame) and then populate the corresponding cell of the data frame
# 5. After all frames have been analyzed, find the average vector value for each column (if a cell is NaN, 0, -1.#ND, etc. it is not included)(every column is the same position between frames)
# 6. Populate the Average_data_frame with the average vector value (row = position, column = vector)

## Part C: Determining the RMS mean velocity field
# 7. Using the total data frame, subtract the mean velocity from each position
# 8. Populate new total_RMS_frame
# 9. Average the columns, and populate the RMS_data_frame

In [125]:
## Part A.1 and A.2 ##

#getting the directory for the stored CSV's for the flow only case (ensure all \ are \\)
csvdir = "C:\\Users\\Harrison\\OneDrive - University of Cincinnati\\Year 5\\Aero Lab\\Lab Work\\Lab 4\\Lab Data\\23-10-20 10.30.02\\Masked Data"

#importing the csv files
csvfiles = glob.glob(os.path.join(csvdir, "*.csv"))

#creating an array to store the imported csv dataframes
dataframes = []

for csvfile in csvfiles:
    df = pd.read_csv(csvfile) #reading each csv file
    dataframes.append(df) #adding the dataframe from the csv into the dataframes array

In [126]:
## Part A.3 & B.4 ##
#importing time to track how long it takes to compute
import time

#creating a dataframe with number of columns equal to the number of data points
#the row is equal to the frame # while the column # is the position of the vector
dt = dataframes[0]
vector_total_frame = pd.DataFrame(columns=np.arange(len(dt)))
u_total_frame = pd.DataFrame(columns=np.arange(len(dt)))
v_total_frame = pd.DataFrame(columns=np.arange(len(dt)))

#initial time
if k/100 == int(k/100):
    time0 = time.time()

for k in range(len(dataframes)):
    dt = dataframes[k]
    vector_values = []
    u_values = []
    v_values = []

    time0 = time.time() #starting time

    #computing the vector for each position
    for i in range(len(dt)):
        vector = np.sqrt((dt.loc[i][2]*(1/1.3499/0.019))**2 + (dt.loc[i][3]*(1/1.3499/0.019))**2) #calculating the vector
        vector_values.append(vector) #adding vector value to array
        u_values.append(dt.loc[i][2]*(1/1.3499/0.019)) #u values
        v_values.append(dt.loc[i][3]*(1/1.3499/0.019)) #v values
        
    #adding the computed vectors to the total_data frame
    vector_total_frame.loc[k] = vector_values 
    u_total_frame.loc[k] = u_values
    v_total_frame.loc[k] = v_values

    if k/100 == int(k/100):
        timef = time.time() #time after computation
        timet = timef - time0
        print("Frame {:.1f} took {:.3f} s to compute".format(k, timet))

Frame 0.0 took 2.543 s to compute
Frame 100.0 took 1.116 s to compute
Frame 200.0 took 1.207 s to compute
Frame 300.0 took 1.160 s to compute
Frame 400.0 took 1.242 s to compute
Frame 500.0 took 1.078 s to compute


In [127]:
#Debugging cell
print(vector_total_frame)

           0           1           2           3           4           5     \
0    115.503363  111.159311  143.484413  189.945175  168.506613  166.037955   
1    160.506155  152.953929  162.821132  167.734193  159.541888  153.919184   
2    157.921555  168.401817  168.299576  184.464478  152.976007  146.551344   
3    159.819952  175.537228  176.271197  174.194942  171.508585  173.780796   
4     31.813676  100.919960  143.853701  154.796973  155.931750  150.217967   
..          ...         ...         ...         ...         ...         ...   
505  113.886192  110.288330  146.440344  175.281067  174.301844  171.161513   
506  145.560902  210.605648  208.919143  182.911480    0.000000  158.470620   
507   33.438096   65.502367  145.385542  196.065738  179.803544  180.309191   
508    0.000000  134.385950  146.261878  151.388475  168.938993  167.612870   
509    0.000000  140.173252  173.471033  163.560448  161.789476  155.362277   

           6           7           8           9   

In [101]:
## Part B.5 and B.6 ##
# Determining the average vector for the cell

#creating the Average_data_frame
Average_data_frame = pd.DataFrame(columns=["x (mm)", "y (mm)", "u mm/s", "v mm/s", "v_avg (mm/s)"])

dt = dataframes[0]

#populating Average_data_frame with the x and y coords
time0 = time.time()
for i in range(len(dt)):
    Average_data_frame.loc[i] = dt.loc[i][0]
    Average_data_frame.loc[i][1] = dt.loc[i][1]
timef = time.time()
print("Positions placed in: {:.2f} s".format(timef-time0))

#calculating the average value
for k in range(len(dt)): #iterating through the columns (each column is a position)
    vector_avg = 0
    u_avg = 0
    v_avg = 0
    v_divider = 0
    u_divider = 0
    vec_divider = 0

    # if k/100 == int(k/100):
    #     time0 = time.time()

    #checking the vectors for if the value is zero or null
    for i in range(len(dataframes)): #iterating the row 
        if vector_total_frame.loc[i][k] != 0: #pd.isnull(total_frame.loc[i][k]) == False:
            vector_avg = vector_avg + vector_total_frame.loc[i][k] #adding the values together
            vec_divider = vec_divider + 1 #number of usable values
        
        if u_total_frame.loc[i][k] != 0:
            u_avg = u_avg + u_total_frame.loc[i][k]
            u_divider = u_divider + 1
        
        if v_total_frame.loc[i][k] != 0:
            v_avg = v_avg + v_total_frame.loc[i][k]
            v_divider = v_divider + 1
        else:
            divider = divider #keeping the divisible number constant
    
    # if avg == 0:
    #     print("Value is zero")
    # else:
    #     cell_avg_vector = avg/divider #calculating the average vector
    
    Average_data_frame.loc[k][2] = u_avg/u_divider
    Average_data_frame.loc[k][3] = v_avg/v_divider
    Average_data_frame.loc[k][4] = vector_avg/vec_divider #adding the average vector to the data frame


    # if k/100 == int(k/100):
    #     timef = time.time()
    #     #print("Average velocities for cell {} calculated in: {:.2f} s".format(k, timef-time0))

print(Average_data_frame)

Positions placed in: 7.28 s
        x (mm)    y (mm)      u mm/s    v mm/s  v_avg (mm/s)
0     -56.4875  109.4688  138.341804  2.988287    139.364727
1     -53.3709  109.4688  148.549197  4.049255    150.263114
2     -50.2544  109.4688  153.874048  4.764621    154.995832
3     -47.1378  109.4688  159.471238  4.437405    160.228526
4     -44.0213  109.4688  163.307461  4.131246    164.043285
...        ...       ...         ...       ...           ...
4656  174.1372  -71.2911  124.685565  0.139076    127.351094
4657  177.2538  -71.2911  126.295639  1.246973    130.160136
4658  180.3703  -71.2911  120.127595  1.173210    125.135449
4659  183.4869  -71.2911  120.391039  1.173647    124.710058
4660  186.6034  -71.2911  101.895336  0.571836    108.062996

[4661 rows x 5 columns]


In [124]:
## Debugging cell
print(vector_total_frame)

           0           1           2           3           4           5     \
0    115.408318  111.067827  132.302211  161.938906  168.368319  165.899126   
1    160.373064  152.826243  162.687430  167.595160  159.410299  153.790825   
2    157.788966  168.261018  168.161064  184.309698  152.847682  146.429060   
3      0.000000  175.393148  176.125807  174.051078  171.367572  173.637213   
4     31.786005  100.835101  143.733437  154.667818  155.800242  150.094542   
..          ...         ...         ...         ...         ...         ...   
505  113.791588  110.196716  146.318732  175.137042  174.157308  171.019917   
506    0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
507   33.411275   65.446368  145.262762  195.902709  179.654532  180.160627   
508    0.000000  134.273331  146.138917  151.261917  168.797004  167.473281   
509    0.000000  140.057607  173.328156  163.423426  161.654661  155.234839   

           6           7           8           9   

In [120]:
## Part C ##
# Creating the RMS dataframe
RMS = pd.DataFrame(index=np.arange(len(dataframes)), columns=np.arange(len(dt)))

#computing the RMS for each cell
for i in range(len(dataframes)): #row of the Mean_RMS
    for j in range(len(vector_total_frame.loc[0][:])): #columns of Mean_RMS
        RMS.loc[i][j] = (vector_total_frame.loc[i][j] - Average_data_frame.loc[j][4])

print(RMS)


           0           1           2           3           4           5     \
0    -23.956409  -39.195287  -22.693621     1.71038    4.325033   -0.953127   
1     21.008337    2.563129    7.691598    7.366634   -4.632987  -13.061428   
2     18.424239   17.997905   13.165231   24.081172  -11.195604  -20.423193   
3   -139.364727   25.130035   21.129975   13.822552    7.324287     6.78496   
4   -107.578722  -49.428012  -11.262396   -5.560708   -8.243044  -16.757711   
..          ...         ...         ...         ...         ...         ...   
505  -25.573139  -40.066398     -8.6771   14.908516   10.114023    4.167664   
506 -139.364727 -150.263114 -154.995832 -160.228526 -164.043285 -166.852253   
507 -105.953452  -84.816746    -9.73307   35.674184   15.611247   13.308373   
508 -139.364727  -15.989783   -8.856915   -8.966608    4.753718    0.621028   
509 -139.364727  -10.205507   18.332324    3.194901   -2.388625  -11.617414   

           6           7           8           9   

In [123]:
#Computing the mean RMS
mean_RMS = pd.DataFrame(columns=["Mean RMS"])

for i in range(len(dataframes)):
    avg = 0
    divider = 0
    
    for j in range(len(RMS)):
        if pd.isnull(RMS.loc[j][i]) == False:
            avg = avg + RMS.loc[i][j]
            divider = divider + 1
        else:
            divider = divider

    RMS_avg = avg/divider
    mean_RMS.loc[i] = RMS_avg

print(mean_RMS)

      Mean RMS
0   -35.401136
1   -30.271169
2   -46.724219
3   -35.751647
4   -31.164411
..         ...
505 -48.714413
506 -58.644971
507 -52.982016
508 -61.674258
509 -82.290313

[510 rows x 1 columns]
